In [1]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b main  # 자기 브랜치 이름으로 변경
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 47 (delta 18), reused 39 (delta 10), pack-reused 0
Unpacking objects: 100% (47/47), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 6.5 MB/s 
     |████████████████████████████████| 272 kB 60.1 MB/s 
     |████████████████████████████████| 2.6 MB 33.3 MB/s 
     |████████████████████████████████| 118 kB 63.7 MB/s 
     |████████████████████████████████| 50 kB 8.6 MB/s 
     |████████████████████████████████| 243 kB 56.4 MB/s 
     |████████████████████████████████| 895 kB 80.8 MB/s 
     |████████████████████████████████| 3.3 MB 49.4 MB/s 
     |████████████████████████████████| 636 kB 55.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 시드 통일하기

In [ ]:
import torch
from transformers import set_seed


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# 모델 불러오기

In [ ]:
# 만약 ForSequenceClassification이 붙은 모델을 사용할 경우
# model_config = {
#     "num_labels": 3,
#     "id2label": {0: 0, 1: 1, 2: 2},
#     "label2id": {0: 0, 1: 1, 2: 2}
# }

# 다음처럼 모델을 불러오는데 사용
# model = AutoModelForSequenceClassification.from_pretrained("모델 이름", **model_config)

# 그냥 모델을 불러오는 경우엔 할 필요 없음

In [ ]:
# 알아서 작성
# from transformers import ...

In [ ]:
# kocharelectra tokenizer를 사용할 경우
# model_max_length값을 직접 입력해줘야함. 설정이 안 돼 있음
# 다른 토크나이저는 설정이 안 돼 있는게 아니면 설정하면 절대 안됨

# from tokenization_kocharelectra import KoCharElectraTokenizer


# tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-small-discriminator", model_max_length=512)

# 하이퍼 파라미터 설정

In [7]:
num_classes = 3

# 원하는 대로 고쳐서 사용
batch_size = 64
lr = 5e-5  # 0.00005
epochs = 15

'0.000050'

In [ ]:
# 옵티마이저와 스케줄러
# 원하는 대로 고쳐서 사용

# from torch.optim import AdamW
# from transformers import AdamW, get_linear_schedule_with_warmup


# LENGTH_OF_TRAIN_DATA = 40242
# num_training_steps = ((LENGTH_OF_TRAIN_DATA - 1) // batch_size + 1) * epochs
# optimizer = AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.1), num_training_steps)

# 만약 이 스케줄러를 사용할 경우,
# optimizer.step() 바로 다음에
# scheduler.step()을 호출해야 함

# f1 score 계산하기

In [ ]:
# 1. torchmetrics 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from torchmetrics import F1


# f1_score = F1(num_classes=num_classes)

# # 검증 단계에서 사용
# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)  # 대충 아웃풋
#     pred = torch.argmax(output, dim=1)
#     batch_f1 = f1_score(pred, labels)
#     print(batch_f1)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 2. datasets 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음
# 3번 사이킷런 사용법 쓰십시오 이건 좋지 않음

# from datasets import load_metric


# f1_score = load_metric("f1")

# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)
#     pred = torch.argmax(output, dim=1)
#     f1_score.add_batch(predictions=pred, references=labels)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 3. sklearn 사용
# import numpy as np
# from sklearn.metrics import accuracy_score, f1_score


# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred, average="micro")

#     return {"accuracy": accuracy, "f1": f1}